In [ ]:
import cv2
import os
import pydicom as dicom
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
###change data format(DICOM->PNG or JPG)###
path='./TCIA_test'
PNG=True
for pat_dir in os.listdir(path):
    for x in os.listdir(os.path.join(path,pat_dir,'CT')):
        #load .dcm file
        dcm_img=dicom.dcmread(os.path.join(path,pat_dir,'CT',x))
        #scale pixel
        pix_arr=dcm_img.pixel_array.astype(float)
        pix_arr_scaled=(np.maximum(pix_arr,0) / pix_arr.max()) * 255.0
        pix_arr_scaled=np.uint8(pix_arr_scaled)
        #change format: .dcm to .png or .jpg
        if PNG == False:
            x = x.replace('.dcm', '.jpg')
        else:
            x = x.replace('.dcm', '.png')
        cv2.imwrite(os.path.join(path,pat_dir,'CT',x),pix_arr_scaled)

In [ ]:
bestImg={}
percentPix={}
for pat_dir in os.listdir(path):
    for x in os.listdir(os.path.join(path,pat_dir,'MASK')):
        _,ext=os.path.splitext(x)
        if ext=='.png' or ext=='.jpg':
            img=cv2.imread(os.path.join(path,pat_dir,'MASK',x),cv2.IMREAD_GRAYSCALE)
        else:
            continue
        numWhitePix = 0 
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if img[i][j]==255:
                    numWhitePix+=1
        percentPix.update({x : (numWhitePix / img.size) * 100.0})
    print(pat_dir,': ',max(percentPix,key=percentPix.get))
    bestImg.update({pat_dir:max(percentPix,key=percentPix.get)})
    percentPix={}